# Open-R1-Distill

- [huggingface/open-r1][2]
- [open-r1/Mixture-of-Thoughts][1]

[1]: https://huggingface.co/datasets/open-r1/Mixture-of-Thoughts
[2]: https://github.com/huggingface/open-r1

## 環境構築

In [ ]:
import logging
import os
import subprocess
import sys

if os.path.exists("debug.log"):
    os.remove("debug.log")

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler("debug.log")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

logger.setLevel(logging.DEBUG)

NVIDIA_SMI = subprocess.run(["nvidia-smi"], capture_output=True, text=True).stdout
logging.info(NVIDIA_SMI)
logging.info(f"Python {sys.version}")

In [ ]:
# 環境変数の設定

import os

os.environ["VLLM_USE_V1"] = "0"
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["LOCAL_RANK"] = "0"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "12345"

In [ ]:
try:
    import google.colab
    IN_COLAB = True
    if not os.path.exists("/content/open-r1"):
        %git clone https://github.com/huggingface/open-r1.git
    %cd /content/open-r1
    %pip install -e ".[dev]" --no-deps
except ImportError:
    IN_COLAB = False
    # !apt update && apt install git-lfs -y
    if not os.path.exists("/workspaces/open-r1-distill/open-r1"):
        %git clone https://github.com/huggingface/open-r1.git
    %cd /workspaces/open-r1-distill/open-r1
    %pip install -e ".[dev]" --no-deps

In [ ]:
# 1) PyTorchとTransformersのインストール
%pip install torch==2.6.0 transformers==4.52.3

# 2) vLLMのインストール
%pip install vllm==0.8.5.post1

# 3) Flash Attentionのインストール
# 2.8.3はundefined symbolエラーが発生するため2.7.3をインストール
# https://github.com/Dao-AILab/flash-attention/issues/1832
%pip install "https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.3/flash_attn-2.7.3+cu12torch2.6cxx11abiFALSE-cp312-cp312-linux_x86_64.whl" --no-build-isolation

# 4) その他の必要なパッケージのインストール
%pip install \
    accelerate==1.4.0 \
    beautifulsoup4 \
    "async-lru" \
    bitsandbytes \
    "distilabel[vllm]" \
    deepspeed==0.16.8 \
    hf_transfer \
    langdetect \
    latex2sympy2_extended \
    liger-kernel \
    "trl[vllm]==0.18.0" \
    math-verify==0.5.2 \
    wandb

## データセット構築

In [ ]:
from datasets import load_dataset
from distilabel.models import vLLM
from distilabel.pipeline import Pipeline
from distilabel.steps.tasks import TextGeneration
import gc
import torch

### データソースの読み込み

In [ ]:
# 72400件の数学データセット
# https://huggingface.co/datasets/AI-MO/NuminaMath-TIR

# テスト用に少しだけ取得
dataset = load_dataset(
    "AI-MO/NuminaMath-TIR",
    split="train",
).select(range(10))

logger.info(f"{len(dataset)=} {dataset[0].keys()=}")

In [ ]:
# problemのサンプルを表示
logger.info(f"problem:\n{dataset[0]['problem']=}")

# (3/5x - 2/y)^8の展開式におけるx^2y^6の係数を分数で求めよ

In [ ]:
# solutionのサンプルを表示
logger.info(f"{dataset[0]['solution']=}")

In [ ]:
# messagesのサンプルを表示
logger.info(f'{dataset[0]["messages"]=}')

### 生成パイプライン作成

In [ ]:
# ステップバイステップで推論し、最終回答を\boxed{}で囲ってください
prompt_template = """\
You will be given a problem. Please reason step by step, and put your final answer within \boxed{}:
{{ instruction }}"""

In [ ]:
# 蒸留パイプラインを作成

# 蒸留元のモデル
# https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

with Pipeline(
    name="distill-qwen-1.5b-r1",
    description="A pipeline to generate data from a distilled r1 model",
) as pipeline:

    # distilabelのvLLMを初期化
    # https://distilabel.argilla.io/dev/components-gallery/llms/vllm/?h=vllm
    llm = vLLM(
        model=model_id,
        tokenizer=model_id,
        extra_kwargs={
            "tensor_parallel_size": 1, # モデル分割のサイズ
            "max_model_len": 1024, # モデルの最大コンテキスト長（8192）
        },
        generation_kwargs={
            "temperature": 0.6, # 生成時の温度パラメータ
            "max_new_tokens": 1024, # 新しく生成するトークンの最大数（8192）
        },
    )

    prompt_column = "problem"

    text_generation = TextGeneration(
        llm=llm, 
        template=prompt_template,
        num_generations=4, # 1つのデータに対して4つ生成
        input_mappings={"instruction": prompt_column} if prompt_column is not None else {}
    )

### 生成

In [ ]:
distiset = pipeline.run(dataset=dataset)

In [ ]:
# 10 * 4 = 40件データが生成される
logger.info(f"{distiset=}")

In [ ]:
# 蒸留元のモデル名
logger.info(f"{distiset['default']['train'][0]['model_name']=}")

In [ ]:
# 生成プロンプト
logger.info(f"{distiset['default']['train'][0]['distilabel_metadata']=}")

In [ ]:
# 生成結果
logger.info(f"{distiset['default']['train'][0]['generation']=}")

# 正解を検証する実装が必要（rejection samplingなど）

## SFT

```sh
python src/open_r1/sft.py \
    --config recipes/OpenR1-Distill-7B/sft/config_distill.yaml \
    --model_name_or_path Qwen/Qwen3-0.6B-Base \
    --hub_model_id OpenR1-Distill-0.6B \
    --output_dir data/OpenR1-Distill-0.6B \
    --push_to_hub False \
    --report_to none
```

In [ ]:
from dataclasses import dataclass, field
from datasets import DatasetDict, concatenate_datasets
from datasets import load_dataset
from functools import partial, update_wrapper
from latex2sympy2_extended import NormalizationConfig
from math_verify import LatexExtractionConfig, parse, verify
from transformers import AutoModelForCausalLM, AutoTokenizer, PreTrainedTokenizer
from transformers import set_seed
from transformers import TrainerCallback
from transformers.trainer_utils import get_last_checkpoint
from transformers.training_args import TrainingArguments
from trl import GRPOTrainer, ModelConfig, TrlParser, get_peft_config
from trl import ModelConfig, get_kbit_device_map, get_quantization_config
from trl import ModelConfig, SFTTrainer, TrlParser, get_peft_config, setup_chat_format
from typing import Any, Callable, Dict, Literal, List, Optional
import asyncio
import datasets
import json
import math
import os
import re
import subprocess
import sys
import torch
import transformers
import trl

logger.info(f"transformers: {transformers.__version__}") # 4.52.3
logger.info(f"torch: {torch.__version__}") # 2.6.0+cu124
logger.info(f"trl: {trl.__version__}") # 0.18.0

### 設定クラス

In [ ]:
@dataclass
class ScriptArguments(trl.ScriptArguments):
    """
    TRLのScriptArgumentsを拡張したクラス
    複数のデータセットを混ぜて、学習に使用するための機能が追加されている
    """

    # Override the dataset_name to make it optional
    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "データセット名。dataset_mixtureが指定されている場合は不要。"}
    )

    dataset_mixture: Optional[dict[str, Any]] = field(
        default=None,
        metadata={"help": "複数のデータセットを混ぜて使用するための設定。シャッフルなどの高度なオプションを含むことができる。"},
    )

    def __post_init__(self):
        """
        初期化後の検証
        """

        if self.dataset_name is None and self.dataset_mixture is None:
            raise ValueError("Either `dataset_name` or `dataset_mixture` must be provided")

        if self.dataset_mixture is not None:
            if not isinstance(self.dataset_mixture, dict) or "datasets" not in self.dataset_mixture:
                raise ValueError(
                    "dataset_mixture must be a dictionary with a 'datasets' key. "
                    "Expected format: {'datasets': [...], 'seed': int}"
                )

            datasets_list = []
            datasets_data = self.dataset_mixture.get("datasets", [])

            if isinstance(datasets_data, list):
                for dataset_config in datasets_data:
                    datasets_list.append(
                        DatasetConfig(
                            id=dataset_config.get("id"),
                            config=dataset_config.get("config"),
                            split=dataset_config.get("split", "train"),
                            columns=dataset_config.get("columns"),
                            weight=dataset_config.get("weight", 1.0),
                        )
                    )
            else:
                raise ValueError("'datasets' must be a list of dataset configurations")

            self.dataset_mixture = DatasetMixtureConfig(
                datasets=datasets_list,
                seed=self.dataset_mixture.get("seed", 0),
                test_split_size=self.dataset_mixture.get("test_split_size", None),
            )

            # Check that column names are consistent across all dataset configs
            columns_sets = [set(dataset.columns) for dataset in datasets_list if dataset.columns is not None]
            if columns_sets:
                first_columns = columns_sets[0]
                if not all(columns == first_columns for columns in columns_sets):
                    raise ValueError(
                        "Column names must be consistent across all dataset configurations in a mixture. "
                        f"Found different column sets: {[list(cols) for cols in columns_sets]}"
                    )

In [ ]:
@dataclass
class SFTConfig(trl.SFTConfig):
    """
    SFTの設定クラス
    TRLのSFTConfigにカスタムの引数を追加している

    - ベンチマーク設定
    - 訓練中のコールバック設定
    - チャットテンプレート設定
    - システムプロンプト設定
    - Hubのリビジョン設定
    - Weights & Biases設定
    """

    benchmarks: list[str] = field(
        default_factory=lambda: [],
        metadata={"help": "トレーニング後に実行するベンチマークのリスト"},
    )

    callbacks: list[str] = field(
        default_factory=lambda: [],
        metadata={"help": "トレーニング中に実行するコールバックのリスト"},
    )

    chat_template: Optional[str] = field(
        default=None, metadata={"help": "使用するチャットテンプレート"}
    )

    system_prompt: Optional[str] = field(
        default=None,
        metadata={"help": "ベンチマークに使用するオプションのシステムプロンプト"},
    )

    hub_model_revision: Optional[str] = field(
        default="main",
        metadata={"help": "モデルをプッシュするHubのブランチ"},
    )

    overwrite_hub_revision: bool = field(
        default=False, metadata={"help": "Hubのリビジョンを上書きするかどうか"}
    )

    push_to_hub_revision: bool = field(
        default=False, metadata={"help": "Hubのリビジョン/ブランチにプッシュするかどうか"}
    )

    wandb_entity: Optional[str] = field(
        default=None,
        metadata={"help": ("実行を保存するエンティティ")},
    )

    wandb_project: Optional[str] = field(
        default=None,
        metadata={"help": ("実行を保存するプロジェクト")},
    )

    wandb_run_group: Optional[str] = field(
        default=None,
        metadata={"help": ("実行を保存するグループ")},
    )

### ユーティリティ関数

In [ ]:
def get_dataset(args: ScriptArguments) -> DatasetDict:
    """
    データセットもしくは混合データセット（mixture of datasets）をダウンロードし初期化する

    Args:
        args (ScriptArguments): データセットの設定を含むスクリプト引数
    Returns:
        DatasetDict: 読み込まれたデータセット
    """

    # 単一データセットの読み込み
    if args.dataset_name and not args.dataset_mixture:
        logger.debug(f"単一データセットを読み込み {args.dataset_name} ")
        return datasets.load_dataset(args.dataset_name, args.dataset_config)

    # 混合データセットの作成
    elif args.dataset_mixture:
        logger.debug(f"混合データセットを作成 {len(args.dataset_mixture.datasets)}")
        seed = args.dataset_mixture.seed
        datasets_list = []

        # 構成要素のデータセットでループ
        for dataset_config in args.dataset_mixture.datasets:
            logger.debug(f"データセットを読み込み {dataset_config.id} (config: {dataset_config.config})")

            # 単一のデータセットを読み込み
            ds = datasets.load_dataset(
                dataset_config.id,
                dataset_config.config,
                split=dataset_config.split,
            )

            # 列が指定されている場合
            if dataset_config.columns is not None:

                # 列を選択
                ds = ds.select_columns(dataset_config.columns)

            # 重みが指定されている場合
            if dataset_config.weight is not None:

                # データセットをシャッフルし、重みに基づいてサブサンプリング
                ds = ds.shuffle(seed=seed).select(
                    range(int(len(ds) * dataset_config.weight))
                )

                logger.debug(
                    f"重み付きでサブサンプリングされたデータセット '{dataset_config.id}' (config: {dataset_config.config}) with weight={dataset_config.weight} to {len(ds)} examples"
                )

            # データセットリストに追加
            datasets_list.append(ds)

        # 正常にデータセットが読み込まれた場合
        if datasets_list:

            # リストを連結して混合データセットを作成
            combined_dataset = concatenate_datasets(datasets_list)

            # 混合データセットをシャッフル
            combined_dataset = combined_dataset.shuffle(seed=seed)
            logger.debug(f"混合データセットを作成 {len(combined_dataset)} examples")

            # テストセットに分割する場合
            if args.dataset_mixture.test_split_size is not None:

                # データセットを訓練とテストに分割
                combined_dataset = combined_dataset.train_test_split(
                    test_size=args.dataset_mixture.test_split_size, seed=seed
                )

                logger.debug(
                    f"データセットを訓練セット {len(combined_dataset['train'])} examples とテストセット {len(combined_dataset['test'])} examples に分割"
                )

                return combined_dataset

            # 分割しない場合
            else:
                return DatasetDict({"train": combined_dataset})
        else:
            raise ValueError("No datasets were loaded from the mixture configuration")

    else:
        raise ValueError("Either `dataset_name` or `dataset_mixture` must be provided")

In [ ]:
def get_tokenizer(model_args: ModelConfig, training_args: Any) -> PreTrainedTokenizer:
    """
    事前学習済みトークナイザーをダウンロードし、初期化する

    Args:
        model_args (ModelConfig): モデル設定
        training_args (Any): トレーニング設定
    Returns:
        PreTrainedTokenizer: 初期化されたトークナイザー
    """
    logger.info(f"Tokenizerを初期化 {model_args.model_name_or_path=}")

    # トークナイザーのロード
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.model_name_or_path,
        revision=model_args.model_revision,
        trust_remote_code=model_args.trust_remote_code,
    )
    logger.debug(f"Tokenizerの設定: {tokenizer}")

    # チャットテンプレートが指定されている場合
    if training_args.chat_template is not None:
        logger.debug(f"チャットテンプレートを設定: {training_args.chat_template}")
        tokenizer.chat_template = training_args.chat_template

    return tokenizer

In [ ]:
def get_model(model_args: ModelConfig, training_args: SFTConfig) -> AutoModelForCausalLM:
    """
    モデルのチェックポイントをダウンロードし、モデルを初期化する

    Args:
        model_args (ModelConfig): モデルの設定
        training_args (SFTConfig, GRPOConfig): トレーニングの設定
    Returns:
        AutoModelForCausalLM: 初期化されたモデル
    """
    logger.info(f"モデルを初期化 {model_args.model_name_or_path}")

    torch_dtype = (
        model_args.torch_dtype \
            if model_args.torch_dtype in ["auto", None] \
            else getattr(torch, model_args.torch_dtype)
    )
    logger.debug(f"{torch_dtype=}")

    quantization_config = get_quantization_config(model_args)
    logger.debug(f"{quantization_config=}")

    model_kwargs = dict(
        revision=model_args.model_revision,
        trust_remote_code=model_args.trust_remote_code,
        attn_implementation=model_args.attn_implementation,
        torch_dtype=torch_dtype,
        use_cache=False if training_args.gradient_checkpointing else True,
        device_map=get_kbit_device_map() if quantization_config is not None else None,
        quantization_config=quantization_config,
    )
    logger.debug(f"{model_kwargs=}")

    # モデルのロード
    model = AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
        **model_kwargs,
    )

    return model

In [ ]:
def get_callbacks(train_config, model_config) -> List[TrainerCallback]:
    """
    学習のチェックポイントが保存されるたびに実行されるコールバック関数を取得する
    新しいリビジョンでチェックポイントを保存するなど
    今回は使わない
    """

    callbacks = []

    # for callback_name in train_config.callbacks:
    #     if callback_name not in CALLBACKS:
    #         raise ValueError(f"Callback {callback_name} not found in CALLBACKS.")
    #     callbacks.append(CALLBACKS[callback_name](model_config))

    return callbacks

In [ ]:
def init_wandb_training(training_args):
    """
    Helper function for setting up Weights & Biases logging tools.
    """
    if training_args.wandb_entity is not None:
        os.environ["WANDB_ENTITY"] = training_args.wandb_entity
    if training_args.wandb_project is not None:
        os.environ["WANDB_PROJECT"] = training_args.wandb_project
    if training_args.wandb_run_group is not None:
        os.environ["WANDB_RUN_GROUP"] = training_args.wandb_run_group

### 訓練設定

In [ ]:
# ベースモデルの設定

model_args = ModelConfig(
    # 本来は"open-r1/Qwen2.5-Math-7B-RoPE-300k"
    # RoPEの周波数を300kに拡張したQwen2.5-Math-7Bモデル
    # SFT中にコンテクスト長を4kから32kに拡張するため
    # https://huggingface.co/open-r1/Qwen2.5-Math-7B-RoPE-300k
    model_name_or_path="Qwen/Qwen3-0.6B-Base",
    model_revision="main",
    torch_dtype="bfloat16",
    attn_implementation="flash_attention_2",
)

In [ ]:
# SFTの訓練設定

training_args = SFTConfig(
    bf16=True,
    do_eval=False,
    eval_strategy="no",
    gradient_accumulation_steps=32, # 本来は"8"だが、メモリ不足を回避するために増加
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    hub_model_id="OpenR1-Distill-0.6B", # "OpenR1-Distill-7B"
    hub_strategy="every_save",
    learning_rate=4.0e-5,
    log_level="info",
    logging_steps=1,
    logging_strategy="steps",
    lr_scheduler_type="cosine_with_min_lr",
    lr_scheduler_kwargs={"min_lr_rate": 0.1},
    packing=False,
    max_grad_norm=0.2,
    max_length=32768, # コンテキスト長を32kに拡張
    max_steps=1, # 本来は"-1"だが、ワークショップ用に少ないステップ数に設定
    num_train_epochs=5, # データセット全体を5順する
    output_dir="data/OpenR1-Distill-0.6B", # "data/OpenR1-Distill-7B"
    overwrite_output_dir=True,
    per_device_eval_batch_size=1,
    per_device_train_batch_size=2,
    push_to_hub=False, # 本来はHubにアップロードするためTrue
    report_to=[], # 本来はログを監視するため["wandb"]
    save_strategy="epoch",
    save_total_limit=1,
    seed=42,
    use_liger_kernel=True,
    warmup_ratio=0.03,
    dataset_num_proc=12,
    eos_token="<|im_end|>",
)

In [ ]:
# データセットの設定

script_args = ScriptArguments(
    # 349,000件のCoTデータセット（code + math + science)
    # https://huggingface.co/datasets/open-r1/Mixture-of-Thoughts
    dataset_name="open-r1/Mixture-of-Thoughts",
    dataset_config="math", # 本来は"all"
)

In [ ]:
# transformersとtorchのシードを設定
set_seed(training_args.seed)

In [ ]:
# 保存済みのチェックポイントがある場合、再開

last_checkpoint = None

if os.path.isdir(training_args.output_dir):
    last_checkpoint = get_last_checkpoint(training_args.output_dir)

if last_checkpoint is not None and training_args.resume_from_checkpoint is None:
    logger.info(f"チェックポイントを検出、{last_checkpoint=} から学習を再開します。")

In [ ]:
# Weights & Biasesが有効な場合、初期化

if "wandb" in training_args.report_to:
    init_wandb_training(training_args)
    logger.info("Weights & Biasesの初期化が完了")

### データセットの読み込み

In [ ]:
dataset = get_dataset(script_args)
logger.info(f"データセットを取得 {dataset=}")

In [ ]:
# ワークショップ用にサブサンプリング

for split in dataset.keys():
    dataset[split] = dataset[split].select(range(1000))

logger.info(f"データセットをサブサンプリング {dataset=}")

### トークナイザーの読み込み

In [ ]:
tokenizer = get_tokenizer(model_args, training_args)
tokenizer

In [ ]:
# チャットテンプレートを設定

if tokenizer.chat_template is None:
    logger.info("No chat template provided, defaulting to ChatML.")
    model, tokenizer = setup_chat_format(model, tokenizer, format="chatml")

tokenizer.chat_template

### モデルの読み込み

In [ ]:
model = get_model(model_args, training_args)
model

### SFTTrainerの作成

In [ ]:
train_dataset = dataset[script_args.dataset_train_split]
logger.info(f"{train_dataset=}")

In [ ]:
eval_dataset = (dataset[script_args.dataset_test_split] \
    if training_args.eval_strategy != "no" else None)
logger.info(f"{eval_dataset=}")

In [ ]:
peft_config = get_peft_config(model_args)
logger.info(f"{peft_config=}")

In [ ]:
callbacks = get_callbacks(training_args, model_args)
logger.info(f"{callbacks=}")

In [ ]:
# SFTTrainerの初期化
# - データセットの前処理とトークン化
# - 正解ラベルの作成とアテンションマスクの作成

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    peft_config=peft_config,
    callbacks=callbacks,
)

### 訓練

In [ ]:
checkpoint = None

if training_args.resume_from_checkpoint is not None:
    checkpoint = training_args.resume_from_checkpoint
elif last_checkpoint is not None:
    checkpoint = last_checkpoint

logger.info(f"訓練の再開に使用するチェックポイント {checkpoint=}")

In [ ]:
# 訓練の実行
# 1. データセットからバッチを取得
# 2. モデルの順伝播を実行し、損失を計算する
# 3. 逆伝播し、勾配を計算する
# 4. 最適化関数を使用してパラメータを更新する
# 定期的に評価したり、チェックポイントを保存したりする

train_result = trainer.train(resume_from_checkpoint=checkpoint)

### 保存

In [ ]:
# 統計情報を抽出
metrics = train_result.metrics

# 「学習に使用したデータ総数」を統計に追加
metrics["train_samples"] = len(dataset[script_args.dataset_train_split])

# コンソールに出力
trainer.log_metrics("train", metrics)

# JSONファイルに保存
trainer.save_metrics("train", metrics)

In [ ]:
# Trainerの内部の状態を保存
# - 最適化関数の状態
# - スケジューラの状態
# - 乱数シードなど
trainer.save_state()

In [ ]:
# モデルの生成設定のEOSトークンIDをトークナイザーのEOSトークンIDに設定
# 無限生成を防止するため
trainer.model.generation_config.eos_token_id = tokenizer.eos_token_id

# 学習済みモデルを保存
trainer.save_model(training_args.output_dir)
logger.info(f"学習済みモデルを保存 {training_args.output_dir}")

In [ ]:
# その他の情報を保存

if trainer.accelerator.is_main_process:

    # モデルカードを作成
    kwargs = {
        "dataset_name": script_args.dataset_name,
        "tags": ["open-r1"],
    }
    trainer.create_model_card(**kwargs)

    # 推論のためにKVキャッシュを有効化し、高速化
    trainer.model.config.use_cache = True

    # モデルの設定を保存（config.json）
    trainer.model.config.save_pretrained(training_args.output_dir)


In [ ]:
# 評価の実行

if training_args.do_eval:
    metrics = trainer.evaluate()
    metrics["eval_samples"] = len(dataset[script_args.dataset_test_split])
    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

In [ ]:
# Hubにプッシュ

if training_args.push_to_hub:
    trainer.push_to_hub(**kwargs)

### メモリ開放

In [ ]:
for target in ["dataset", "tokenizer", "model", "trainer", "trainer_result", "mertics"]:
    try:
        del globals()[target]
    except:
        pass

import gc
gc.collect()
torch.cuda.empty_cache()

!nvidia-smi

## GRPO

```sh
!RANK=0 WORLD_SIZE=1 LOCAL_RANK=0 MASTER_ADDR=localhost MASTER_PORT=12345 python src/open_r1/grpo.py \
    --config recipes/DeepSeek-R1-Distill-Qwen-1.5B/grpo/config_demo.yaml \
    --model_name_or_path Qwen/Qwen3-0.6B-Base \
    --hub_model_id OpenR1-Distill-0.6B \
    --output_dir data/OpenR1-Distill-0.6B \
    --push_to_hub False \
    --report_to none \
    --vllm_mode colocate \
    --per_device_train_batch_size 1 \
    --num_generations 4 \
    --gradient_accumulation_steps 8
```

In [ ]:
from dataclasses import dataclass, field
from datasets import DatasetDict, concatenate_datasets
from datasets import load_dataset
from functools import partial, update_wrapper
from latex2sympy2_extended import NormalizationConfig
from math_verify import LatexExtractionConfig, parse, verify
from transformers import AutoModelForCausalLM, AutoTokenizer, PreTrainedTokenizer
from transformers import set_seed
from transformers import TrainerCallback
from transformers.trainer_utils import get_last_checkpoint
from transformers.training_args import TrainingArguments
from trl import GRPOTrainer, ModelConfig, TrlParser, get_peft_config
from trl import ModelConfig, get_kbit_device_map, get_quantization_config
from trl import ModelConfig, SFTTrainer, TrlParser, get_peft_config, setup_chat_format
from typing import Any, Callable, Dict, Literal, List, Optional
import asyncio
import datasets
import json
import math
import os
import re
import subprocess
import sys
import transformers
import trl
import torch

logger.info(f"transformers: {transformers.__version__}") # 4.52.3
logger.info(f"torch: {torch.__version__}") # 2.6.0+cu124
logger.info(f"trl: {trl.__version__}") # 0.18.0

### 設定クラス

#### GRPOConfig

In [ ]:
@dataclass
class GRPOConfig(trl.GRPOConfig):
    """
    GRPOの設定クラス
    TRLのGRPOConfigにカスタムの引数を追加している
    """

    benchmarks: list[str] = field(
        default_factory=lambda: [],
        metadata={"help": "訓練後のベンチマークを実行するためのリスト"},
    )

    callbacks: list[str] = field(
        default_factory=lambda: [],
        metadata={"help": "訓練中に実行するコールバックのリスト"},
    )

    chat_template: Optional[str] = field(default=None, metadata={
        "help": "使用するチャットテンプレート"
    })

    hub_model_revision: Optional[str] = field(
        default="main", metadata={"help": "モデルをプッシュするHubのブランチ"}
    )

    num_completions_to_print: int = field(default=0, metadata={
        "help": "表示する完了数"
    })

    overwrite_hub_revision: bool = field(default=False, metadata={
        "help": "Hubのリビジョンを上書きするかどうか"
    })

    push_to_hub_revision: bool = field(default=False, metadata={
        "help": "Hubのリビジョン/ブランチにプッシュするかどうか"
    })

    system_prompt: Optional[str] = field(
        default=None,
        metadata={"help": "使用するオプションのシステムプロンプト"},
    )

    wandb_log_unique_prompts: bool = field(
        default=True,
        metadata={
            "help": "ユニークなプロンプトをwandbにログするかどうか。各ユニークなプロンプトごとに新しいランが作成されます。"
        },
    )
    wandb_entity: Optional[str] = field(
        default=None,
        metadata={"help": "ランを保存するエンティティ"},
    )

    wandb_project: Optional[str] = field(
        default=None,
        metadata={"help": "ランを保存するプロジェクト"},
    )

    wandb_run_group: Optional[str] = field(
        default=None,
        metadata={"help": "ランを保存するグループ"},
    )

#### GRPOScriptArguments

In [ ]:
@dataclass
class GRPOScriptArguments(ScriptArguments):
    """
    GRPOの訓練設定
    """

    reward_funcs: list[str] = field(
        default_factory=lambda: ["accuracy", "format", "tag_count"],
        metadata={
            "help": (
                "報酬関数のリスト。可能な値: 'accuracy'（正確性）、'format'（フォーマット）、'reasoning_steps'（推論ステップ数）、'cosine'（コサイン類似度）、'repetition_penalty'（繰り返しペナルティ）、'length'（長さ）、'tag_count'（タグ数）、'code'（コード）、'code_format'（コードフォーマット）"
            )
        },
    )

    cosine_min_value_wrong: float = field(
        default=0.0,
        metadata={"help": "間違えた回答に対する最小報酬"},
    )

    cosine_max_value_wrong: float = field(
        default=-0.5,
        metadata={"help": "間違えた回答に対する最大報酬"},
    )

    cosine_min_value_correct: float = field(
        default=0.5,
        metadata={"help": "正しい回答に対する最小報酬"},
    )

    cosine_max_value_correct: float = field(
        default=1.0,
        metadata={"help": "正しい回答に対する最大報酬"},
    )

    cosine_max_len: int = field(
        default=1000,
        metadata={"help": "スケーリングの最大長"},
    )

    repetition_n_grams: int = field(
        default=3,
        metadata={"help": "n-gramの長さを繰り返しペナルティ報酬に使用"},
    )

    repetition_max_penalty: float = field(
        default=-1.0,
        metadata={"help": "繰り返しペナルティ報酬の最大値"},
    )

    code_language: str = field(
        default="python",
        # '(?:python|cpp)'
        metadata={
            "help": "コードフォーマット報酬のための言語。E2Bがサポートする言語に基づく https://e2b.dev/docs/code-interpreting/supported-languages",
            "choices": ["python", "javascript", "r", "java", "bash", "cpp"],
        },
    )
    code_eval_test_batch_size: int = field(
        default=1,
        metadata={
            "help": "各生成に対して、これだけの数のテストケースを並列で評価し、そのうちのいずれかが失敗（スコア0）した場合は評価を停止し、そうでなければ次のテストケースのバッチに進む。評価サーバーの過負荷を避け、誤った解答に対する時間を節約するのに有用。"
        },
    )

    code_eval_scoring_mode: Literal["pass_fail", "partial", "weighted_sum"] = field(
        default="weighted_sum",
        metadata={"help": "テストケースの合格割合を報酬として使用します。falseの場合は0/1のスコアリングを使用します。"},
    )

    parallel_code_exec_per_proc: int = field(
        default=2,
        metadata={
            "help": "プロセスごとの並列E2Bコード実行数。デフォルトの2は、8 GPUを使用したE2BのFree Hobbyティアに適しています。"
        },
    )

    dataset_prompt_column: str = field(
        default="prompt",
        metadata={"help": "トレーニングに使用するプロンプトの列名。"},
    )

    e2b_router_url: Optional[str] = field(
        default=None,
        metadata={"help": "E2BルーターのURL。scripts/e2b_router.pyを参照してください"},
    )

    morph_router_url: Optional[str] = field(
        default=None,
        metadata={"help": "MorphCloudルーターのURL。scripts/morph_router.pyを参照してください"},
    )

    code_provider: Optional[str] = field(
        default="e2b",
        metadata={
            "help": "コード実行のプロバイダー。オプション: 'e2b', 'local', 'morph'",
            "choices": ["e2b", "local", "morph"],
        },
    )

    ioi_provider: Optional[str] = field(
        default="piston",
        metadata={
            "help": "IOIコード実行のプロバイダー。オプション: 'piston', 'morph'",
            "choices": ["piston", "morph"],
        },
    )

    max_completion_len: int = field(
        default=16384,
        metadata={"help": "補完の最大文字数"},
    )

    soft_punish_cache: int = field(
        default=4096,
        metadata={"help": "補完の最小文字数"},
    )

### 報酬関数

In [ ]:
def accuracy_reward(completions: list[list[dict[str, str]]], solution: list[str], **kwargs) -> list[Optional[float]]:
    """
    モデルが生成した回答が、正解と数学的に一致しているかを判定し、報酬を与える関数

    Args:
        completions (list[list[dict[str, str]]]): モデルが生成した回答のリスト
        solution (list[str]): 正解のリスト
    Returns:
        list[Optional[float]]: 各回答に対する報酬のリスト
    """

    # 1) 生成したテキスト(completion)と正解データ(solution)を受け取る

    contents = [completion[0]["content"] for completion in completions]

    rewards = []

    # 2) 各回答について、正解と数学的に一致しているかを検証し、報酬を計算する

    for content, sol in zip(contents, solution):

        # 正解データから正解を抽出
        gold_parsed = parse(
            sol,
            extraction_mode="first_match",
        )

        # 解析可能な正解が存在する場合
        if len(gold_parsed) != 0:

            # 回答は正しいLaTeX形式で提供される必要がある（不正な演算子は不可）
            # math_verifyのparse関数を使用して、回答から数学的表現を抽出
            answer_parsed = parse(
                content,
                extraction_config=[
                    LatexExtractionConfig(
                        normalization_config=NormalizationConfig(
                            nits=False,
                            malformed_operators=False,
                            basic_latex=True,
                            equations=True,
                            boxed="all", # \boxed{}内の式を優先的に抽出
                            units=True,
                        ),
                        # Ensures that boxed is tried first
                        boxed_match_priority=0,
                        try_extract_without_anchor=False,
                    )
                ],
                extraction_mode="first_match",
            )

            try:
                # math_verifyのverify関数を使用して、回答と正解が数学的に一致しているかを検証
                reward = float(verify(gold_parsed, answer_parsed))
            except Exception as e:
                # 失敗した場合は報酬をNoneに設定し、スキップ
                print(f"verify failed: {e}, answer: {answer_parsed}, gold: {gold_parsed}")
                reward = None

        # 解析可能な正解が存在しない場合
        else:
            # スキップするため明示的にNoneを割り当てる
            reward = None
            print("Failed to parse gold solution: ", sol)

        rewards.append(reward)

    return rewards

accuracy_reward(
    completions=[
        [{"role": "assistant", "content": "Answer is \\boxed{42}"}],
    ], 
    solution=["42"]
)

In [ ]:
def format_reward(completions, **kwargs):
    """
    思考過程が<think>と</think>タグで囲まれ、最終回答が<answer>と</answer>タグで
    囲まれているかを確認する報酬関数

    Args:
        completions (list[list[dict[str, str]]]): モデルが生成した回答のリスト
    Returns:
        list[float]: 各回答に対する報酬のリスト
        （フォーマットが正しい場合は1.0、そうでない場合は0.0）
    # """

    # 正規表現パターンを定義
    pattern = r"^<think>\n.*?\n</think>\n<answer>\n.*?\n</answer>$"

    # contentキーの値を抽出
    completion_contents = [completion[0]["content"] for completion in completions]

    # 各回答がパターンにマッチするかを確認
    matches = [
        re.match(pattern, content, re.DOTALL | re.MULTILINE)
        for content in completion_contents
    ]

    # マッチ結果に基づいて報酬を計算
    return [1.0 if match else 0.0 for match in matches]

format_reward(
    completions=[[
        {
            "role": "assistant",
            "content": "<think>\nHold on...\n</think>\n<answer>\n42\n</answer>"
        }
    ]], 
)

In [ ]:
def tag_count_reward(completions, **kwargs) -> list[float]:
    """
    format_reward()に関連する<think>および<answer>タグの所望の数が生成されているかの報酬
    """

    # 採点する関数
    def count_tags(text: str) -> float:
        count = 0.0

        if text.count("<think>\n") == 1:
            count += 0.25

        if text.count("\n</think>\n") == 1:
            count += 0.25

        if text.count("\n<answer>\n") == 1:
            count += 0.25

        if text.count("\n</answer>") == 1:
            count += 0.25

        return count

    # contentキーの値を抽出
    contents = [completion[0]["content"] for completion in completions]

    # 各回答に対してタグの数をカウント
    return [count_tags(c) for c in contents]

tag_count_reward(
    completions=[[
        {
            "role": "assistant",
            "content": "<think>\nHold on...\n</think>\n<answer>\n42\n"
        }
    ]], 
)

In [ ]:
def get_reward_funcs(script_args) -> list[Callable]:
    """
    指定された報酬関数のリストを取得する

    Args:
        script_args: スクリプトの引数
    Returns:
        list[Callable]: 報酬関数のリスト
    """

    REWARD_FUNCS_REGISTRY = {
        "accuracy": accuracy_reward,
        "format": format_reward,
        # "reasoning_steps": reasoning_steps_reward,
        # "cosine": get_cosine_scaled_reward(
        #     min_value_wrong=script_args.cosine_min_value_wrong,
        #     max_value_wrong=script_args.cosine_max_value_wrong,
        #     min_value_correct=script_args.cosine_min_value_correct,
        #     max_value_correct=script_args.cosine_max_value_correct,
        #     max_len=script_args.cosine_max_len,
        # ),
        # "repetition_penalty": get_repetition_penalty_reward(
        #     ngram_size=script_args.repetition_n_grams,
        #     max_penalty=script_args.repetition_max_penalty,
        # ),
        # "length": len_reward,
        # "code": update_wrapper(
        #     partial(
        #         code_reward,
        #         num_parallel=script_args.parallel_code_exec_per_proc,
        #         provider_type=script_args.code_provider,
        #         enforce_same_language=getattr(script_args, "enforce_same_language", False),
        #     ),
        #     code_reward,
        # ),
        # "binary_code": update_wrapper(
        #     partial(
        #         binary_code_reward,
        #         num_parallel=script_args.parallel_code_exec_per_proc,
        #         provider_type=script_args.code_provider,
        #         enforce_same_language=getattr(script_args, "enforce_same_language", False),
        #     ),
        #     binary_code_reward,
        # ),
        # "ioi_code": update_wrapper(
        #     partial(
        #         ioi_code_reward,
        #         test_batch_size=script_args.code_eval_test_batch_size,
        #         provider_type=getattr(script_args, "ioi_provider", "piston"),
        #     ),
        #     ioi_code_reward,
        # ),
        # "cf_code": update_wrapper(
        #     partial(
        #         cf_code_reward,
        #         test_batch_size=script_args.code_eval_test_batch_size,
        #         scoring_mode=script_args.code_eval_scoring_mode,
        #     ),
        #     cf_code_reward,
        # ),
        # "code_format": get_code_format_reward(language=script_args.code_language),
        "tag_count": tag_count_reward,
        # "soft_overlong_punishment": get_soft_overlong_punishment(
        #     max_completion_len=script_args.max_completion_len,
        #     soft_punish_cache=script_args.soft_punish_cache,
        # ),
    }
    reward_funcs = [REWARD_FUNCS_REGISTRY[func] for func in script_args.reward_funcs]

    return reward_funcs


### 訓練設定

In [ ]:
# データセットと報酬関数の設定

script_args = GRPOScriptArguments(
    # https://huggingface.co/datasets/open-r1/OpenR1-Math-220k
    dataset_name="open-r1/OpenR1-Math-220k",
    dataset_prompt_column="problem",
    reward_funcs=["accuracy", "format", "tag_count"],
)

In [ ]:
# GRPOの訓練設定

training_args = GRPOConfig(
    bf16=True,
    use_vllm=True,
    do_eval=False,
    gradient_accumulation_steps=16, # 4から16に増加。 メモリ不足を回避するため
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    # hub_model_id="DeepSeek-R1-Distill-Qwen-1.5B-GRPO",
    hub_model_id="DeepSeek-R1-Distill-Qwen-0.6B-GRPO",
    hub_strategy="every_save",
    learning_rate=1.0e-6, # SFTよりも低い学習率
    log_completions=True,
    log_level="info",
    logging_first_step=True,
    logging_steps=1,
    logging_strategy="steps",
    lr_scheduler_type="cosine_with_min_lr",
    lr_scheduler_kwargs={"min_lr_rate": 0.1},
    max_prompt_length=512,
    max_completion_length=2048,
    max_steps=1, # 本来は-1 だが、ワークショップ用に少ないステップ数に設定
    num_generations=4, # 16から4に減少。 メモリ不足を回避するため
    num_train_epochs=1, # 1エポックのみ
    # output_dir="data/DeepSeek-R1-Distill-Qwen-1.5B-GRPO",
    output_dir="data/DeepSeek-R1-Distill-Qwen-0.6B-GRPO",
    overwrite_output_dir=True,
    per_device_eval_batch_size=1, # 本来は16だが、メモリ不足を回避するため1に設定
    per_device_train_batch_size=1, # 本来は16だが、メモリ不足を回避するため1に設定
    push_to_hub=False, # 本来はHubにアップロードするためTrue
    report_to=[], # 本来はログを監視するため["wandb"]
    reward_weights=[1.0, 1.0, 1.0], # 報酬関数の重み
    save_strategy="epoch",
    save_total_limit=1,
    seed=42,
    temperature=0.7,
    use_liger_kernel=True,
    warmup_ratio=0.1,
    system_prompt="You are a helpful AI Assistant that provides well-reasoned and detailed responses. You first think about the reasoning process as an internal monologue and then provide the user with the answer. Respond in the following format: <think>\n...\n</think>\n<answer>\n...\n</answer>",
    vllm_mode="colocate",
)

In [ ]:
model_args = ModelConfig(
    # SFTで訓練済みのモデルを指定する
    # model_name_or_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    model_name_or_path="Qwen/Qwen3-0.6B-Base",
    model_revision="main",
    torch_dtype="bfloat16",
    attn_implementation="flash_attention_2",
)

In [ ]:
set_seed(training_args.seed)

In [ ]:
# 訓練済みのチェックポイントがある場合、再開

last_checkpoint = None

if os.path.isdir(training_args.output_dir):
    last_checkpoint = get_last_checkpoint(training_args.output_dir)

if last_checkpoint is not None and training_args.resume_from_checkpoint is None:
    logger.info(f"チェックポイントを検出、{last_checkpoint=} から学習を再開します。")

In [ ]:
# Weights & Biasesが有効な場合、初期化

if "wandb" in training_args.report_to:
    init_wandb_training(training_args)
    logger.info("Weights & Biasesの初期化が完了")

### データセットの読み込み

In [ ]:
# Load the dataset
dataset = get_dataset(script_args)
dataset

logger.info(f"データセットを取得 {dataset=}")
# 93733

In [ ]:
# デバッグ用に1000サンプルに制限

for split in dataset.keys():
    dataset[split] = dataset[split].select(range(1000))

logger.info(f"データセットをサブサンプリング {dataset=}")

In [ ]:
def make_conversation(example, prompt_column: str = script_args.dataset_prompt_column):
    """
    データセットの各例をチャット形式に変換し、promptフィールドを作成する関数
    """
    prompt = []

    # システムプロンプトが指定されている場合、最初に追加
    if training_args.system_prompt is not None:
        prompt.append({"role": "system", "content": training_args.system_prompt})

    # problemカラムが存在しない場合、エラーをスロー
    if prompt_column not in example:
        raise ValueError(f"Dataset Question Field Error: {prompt_column} is not supported.")

    # problemカラムの内容をユーザーメッセージとして追加
    prompt.append({"role": "user", "content": example[prompt_column]})

    # promptフィールドを返す
    return {"prompt": prompt}

dataset = dataset.map(make_conversation)
logger.info(f"チャット形式に変換されたデータセットを取得 {dataset=}")

In [ ]:
# messagesカラムを削除
for split in dataset:
    if "messages" in dataset[split].column_names:
        dataset[split] = dataset[split].remove_columns("messages")

logger.info(f"messagesカラムを削除したデータセットを取得 {dataset=}")

### トークナイザーの読み込み

In [ ]:
# トークナイザーを読み込み

tokenizer = get_tokenizer(model_args, training_args)
logger.info(f"トークナイザーを取得 {tokenizer=}")

### モデルの読み込み

In [ ]:
# SFT済みモデルを読み込み

model = get_model(model_args, training_args)
logger.info(f"モデルを取得 {model=}")

### 報酬関数の読み込み

In [ ]:
reward_funcs = get_reward_funcs(script_args)
logger.info(f"報酬関数を取得 {reward_funcs=}")

### GRPOTrainerの作成

In [ ]:
train_dataset = dataset[script_args.dataset_train_split]
logger.info(f"{train_dataset=}")

In [ ]:
eval_dataset  =(
    dataset[script_args.dataset_test_split]
    if training_args.eval_strategy != "no" else None
)
logger.info(f"{eval_dataset=}")

In [ ]:
peft_config = get_peft_config(model_args)
logger.info(f"{peft_config=}")

In [ ]:
callbacks = get_callbacks(training_args, model_args)
logger.info(f"{callbacks=}")

In [ ]:
trainer = GRPOTrainer(
    model=model,
    reward_funcs=reward_funcs,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    callbacks=callbacks,
    processing_class=tokenizer,
)

In [ ]:
checkpoint = None

if training_args.resume_from_checkpoint is not None:
    checkpoint = training_args.resume_from_checkpoint

elif last_checkpoint is not None:
    checkpoint = last_checkpoint

logger.info(f"訓練の再開に使用するチェックポイント {checkpoint=}")

### 訓練

In [ ]:
train_result = trainer.train(resume_from_checkpoint=checkpoint)

### 保存

In [ ]:
# 統計情報を抽出
metrics = train_result.metrics

# 「学習に使用したデータ総数」を統計に追加
metrics["train_samples"] = len(dataset[script_args.dataset_train_split])

# コンソールに出力
trainer.log_metrics("train", metrics)

# JSONファイルに保存
trainer.save_metrics("train", metrics)

# Trainerの内部の状態を保存
trainer.save_state()

In [ ]:
# モデルの生成設定のEOSトークンIDをトークナイザーのEOSトークンIDに設定
trainer.model.generation_config.eos_token_id = tokenizer.eos_token_id

# 学習済みモデルを保存
trainer.save_model(training_args.output_dir)

# その他の情報を保存
if trainer.accelerator.is_main_process:

    # モデルカードを作成
    kwargs = {
        "dataset_name": script_args.dataset_name,
        "tags": ["open-r1"],
    }
    trainer.create_model_card(**kwargs)

    # 推論のためにKVキャッシュを有効化し、高速化
    trainer.model.config.use_cache = True

    # モデルの設定を保存（config.json）
    trainer.model.config.save_pretrained(training_args.output_dir)

In [ ]:
# 評価の実行

if training_args.do_eval:
    metrics = trainer.evaluate()
    metrics["eval_samples"] = len(dataset[script_args.dataset_test_split])
    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

In [ ]:
# Hubにプッシュ

if training_args.push_to_hub:
    logger.info("Pushing to hub...")
    trainer.push_to_hub(**kwargs)